In [1]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
import json
import pprint
import tensorflow as tf
import matplotlib.pyplot as plt
from os.path import expanduser

print("Done")

Done


In [2]:
"""
## Define TFRecords helper functions
"""


def image_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[tf.io.encode_png(value).numpy()])
    )


def bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))


def float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def float_feature_list(value):
    """Returns a list of float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))


def create_example(image, path, example):
    feature = {
        "image": image_feature(image),
        "path": bytes_feature(path),
        "area": float_feature(example["area"]),
        "bbox": float_feature_list(example["bbox"]),
        "category_id": int64_feature(example["category_id"]),
        "id": int64_feature(example["id"]),
        "image_id": int64_feature(example["image_id"]),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))


def parse_tfrecord_fn(example):
    feature_description = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "path": tf.io.FixedLenFeature([], tf.string),
        "area": tf.io.FixedLenFeature([], tf.float32),
        "bbox": tf.io.VarLenFeature(tf.float32),
        "category_id": tf.io.FixedLenFeature([], tf.int64),
        "id": tf.io.FixedLenFeature([], tf.int64),
        "image_id": tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, feature_description)
    example["image"] = tf.io.decode_png(example["image"], channels=3)
    example["bbox"] = tf.sparse.to_dense(example["bbox"])
    return example

print("Done")

Done


In [ ]:
"""
The images are a collection of PNG files and the meta-data are stored in a JSON file
which, according to the [official site](https://cocodataset.org/#format-data),
contains the following properties:

```
id: int,
image_id: int,
category_id: int,
segmentation: RLE or [polygon], object segmentation mask
bbox: [x,y,width,height], object bounding box coordinates
area: float, area of the bounding box
iscrowd: 0 or 1, is single object or a collection
```
"""
home = expanduser("~")
train_augmented_fp = home + "/OneDrive/Documents/Github/GEORGE/Dataset/Custom_Dataset/Train"
val_augmented_fp = home + "/OneDrive/Documents/Github/GEORGE/Dataset/Custom_Dataset/Validation"
test_augmented_fp = home + "/OneDrive/Documents/Github/GEORGE/Dataset/Custom_Dataset/Test"
tfrecords_dir = home + "/OneDrive/Documents/Github/GEORGE/Dataset/Custom_Dataset/tfrecords"

if not os.path.exists(tfrecords_dir):
    os.makedirs(tfrecords_dir)  # creating TFRecords output folder

fps = [train_augmented_fp, val_augmented_fp, test_augmented_fp]

for fp in fps:
    current_tf_dir = tfrecords_dir + "/" + fp.split("/")[-1]
    if not os.path.exists(current_tf_dir):
        os.makedirs(current_tf_dir)  # creating TFRecords output folder
    images_dir = fp
    annotations_dir = fp
    annotation_file = os.path.join(fp, "custom_bee_dataset.json")
    
    with open(annotation_file, "r") as f:
        annotations = json.load(f)["annotations"]
    
    print(f"Number of images: {len(annotations)}")
    
    pprint.pprint(annotations[60])
    
    """
    ## Parameters
    
    `num_samples` is the number of data samples on each TFRecord file.
    
    `num_tfrecords` is total number of TFRecords that we will create.
    """
    
    num_samples = 4096
    num_tfrecords = len(annotations) // num_samples
    if len(annotations) % num_samples:
        num_tfrecords += 1  # add one record if there are any remaining samples

    """
    ## Generate data in the TFRecord format
    
    Let's generate the COCO2017 data in the TFRecord format. The format will be
    `file_{number}.tfrec` (this is optional, but including the number sequences in the file
    names can make counting easier).
    """
    
    for tfrec_num in range(num_tfrecords):
        samples = annotations[(tfrec_num * num_samples) : ((tfrec_num + 1) * num_samples)]
    
        with tf.io.TFRecordWriter(
            current_tf_dir + "/file_%.2i-%i.tfrec" % (tfrec_num, len(samples))
        ) as writer:
            for sample in samples:
                image_path = f"{images_dir}/{sample['image_id']:01d}.png"
                image = tf.io.decode_png(tf.io.read_file(image_path))
                example = create_example(image, image_path, sample)
                writer.write(example.SerializeToString())
    
    """
    ## Explore one sample from the generated TFRecord
    """
    
    raw_dataset = tf.data.TFRecordDataset(f"{current_tf_dir}/file_00-{num_samples}.tfrec")
    parsed_dataset = raw_dataset.map(parse_tfrecord_fn)
    
    for features in parsed_dataset.take(1):
        for key in features.keys():
            if key != "image":
                print(f"{key}: {features[key]}")
    
        print(f"Image shape: {features['image'].shape}")
        plt.figure(figsize=(7, 7))
        plt.imshow(features["image"].numpy())
        plt.show()

Number of images: 1049869
{'area': 6300.0,
 'bbox': [259.0, 536.0, 70.0, 90.0],
 'category_id': 3,
 'id': 61,
 'image_id': 4,
 'iscrowd': 0,
 'segmentation': None}
